In [2]:
import pandas as pd
import numpy as np
import statistics
import requests
import datetime


# 데이터 저장 폴더
root_dir = "./bs_analysis/"

list_ordered_key = ["CD", "DT"]
list_using_columns = [
    "NM", "FRGN_RT", "END_PRC", "VOL", "REMAIN_BUY", "REMAIN_SELL", "REMAIN_BUY_AMOUNT",
    "PRC_AVG_5", "VOL_AVG_5", "PRC_AVG_10", "VOL_AVG_10", "PRC_MEDIAN", "VOL_MEDIAN", "DAY_SUMMARY"
]
list_columns = [
    "CD", "NM",
    "LAST_FRGN_RT", "LAST_END_PRC", "PRC_MEDIAN", "LAST_PRC_AVG_5", "LAST_PRC_AVG_10",
    "LAST_VOL", "VOL_MEDIAN",
    "ROE", "ROE_AVG", "PER", "PBR", "PROFIT",
    "FRGN_RT", "END_PRC", "VOL", "REMAIN_DEAL"
]
dict_rename_cols = {
    "CD": "코드",
    "NM": "종목",
    "LAST_FRGN_RT": "최근 외국인 지분율",
    "LAST_END_PRC": "최근 종가", 
    "PRC_MEDIAN": "최근10일 종가 중간값", 
    "LAST_PRC_AVG_5": "최근5일 종가 평균", 
    "LAST_PRC_AVG_10": "최근10일 종가 평균",
    "LAST_VOL": "마지막 거래량", 
    "VOL_MEDIAN": "최근10일 거래량 중간값",
    "ROE": "자기자본이익률(ROE)", 
    "ROE_AVG": "ROE 평균", 
    "PER": "주가수익비율", 
    "PBR": "주가순자산비율", 
    "PROFIT": "영업이익",
    "FRGN_RT": "외국인지분율", 
    "END_PRC": "종가", 
    "VOL": "거래량", 
    "REMAIN_DEAL": "매수잔액",
}
# 작업일자
dt = datetime.datetime.now().strftime("%Y-%m-%d")

In [19]:
df_calc_base = pd.read_csv(f"{root_dir}day_summary.csv", encoding="utf-8-sig")[list_ordered_key + list_using_columns]
df_calc_base["REMAIN_DEAL"] = df_calc_base["REMAIN_BUY_AMOUNT"]

df_calc_base

,CD,DT,NM,FRGN_RT,END_PRC,VOL,REMAIN_BUY,REMAIN_SELL,REMAIN_BUY_AMOUNT,PRC_AVG_5,VOL_AVG_5,PRC_AVG_10,VOL_AVG_10,PRC_MEDIAN,VOL_MEDIAN,DAY_SUMMARY,REMAIN_DEAL
0,A098120,2022-03-25,마이크로컨텍솔,14.89,11050,76436,15439,27759,-29.0,10890,57262,10541,63758,10675,60328,"{'시가': '11150', '고가': '11200', '저가': '10850', ...",-29.0
1,A095570,2022-03-25,AJ네트웍스,4.87,6030,83139,11125,10822,20.0,5900,153172,5765,109332,5715,64439,"{'시가': '6100', '고가': '6130', '저가': '5970', '52...",20.0
2,A282330,2022-03-25,BGF리테일,30.96,181000,32534,5453,10789,-740.0,179600,33326,177350,46458,178500,38709,"{'시가': '180000', '고가': '182500', '저가': '178500...",-740.0
3,A000995,2022-03-25,DB하이텍1우,1.07,94700,171,423,242,13.0,94680,136,93790,275,93900,200,"{'시가': '95000', '고가': '95000', '저가': '94700', ...",13.0
4,A000990,2022-03-25,DB하이텍,24.09,76000,998870,62391,29007,1812.0,74520,1046020,72130,881717,72200,787426,"{'시가': '78400', '고가': '78500', '저가': '75700', ...",1812.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,A298020,2022-03-21,효성티앤씨,11.90,459000,59036,5742,1459,NaN,471100,43878,475800,38211,478250,36263,NaN,NaN
1451,A298000,2022-03-21,효성화학,7.27,256500,21361,3863,595,NaN,262200,15565,263800,14772,264250,13030,NaN,NaN
1452,A081660,2022-03-21,휠라홀딩스,41.53,33150,332995,26707,6677,NaN,33470,283135,33580,391033,33675,340460,NaN,NaN
1453,A205470,2022-03-21,휴마시스,5.66,18550,1735532,157242,82334,NaN,19050,4291294,18725,2961532,18725,1713841,NaN,NaN


In [22]:
base_day = 5
dict_col_index = {}
list_cols = []
list_no_target_columns = ["CD", "NM", "DT", "DAY_SUMMARY"]
idx = 0
for col in df_calc_base.columns:
    if col in list_no_target_columns: continue
    list_cols.append(col)
    dict_col_index[col] = idx
    idx += 1
# 종목
list_nm = df_calc_base["NM"].unique()
# 추세 딕셔너리
dict_rising = {}
# 딕셔너리 데이터 생성
for nm in list_nm:
    list_dict = []
    for col in list_cols:
        dict_col_list_val = {}
        list_val = list(df_calc_base[df_calc_base["NM"] == nm][["NM", col]].head(base_day).T.reset_index().loc[1])[1:]
        dict_col_list_val[col] = list_val
        list_dict.append(dict_col_list_val)
    dict_rising[nm] = list_dict

dict_rising['마이크로컨텍솔']

[{'FRGN_RT': [14.89, 14.84, 14.71]},
 {'END_PRC': [11050, 11050, 10950]},
 {'VOL': [76436, 60821, 78976]},
 {'REMAIN_BUY': [15439, 25942, 18130]},
 {'REMAIN_SELL': [27759, 35191, 21199]},
 {'REMAIN_BUY_AMOUNT': [-29.0, nan, nan]},
 {'PRC_AVG_5': [10890, 10810, 10710]},
 {'VOL_AVG_5': [57262, 53828, 59011]},
 {'PRC_AVG_10': [10541, 10476, 10411]},
 {'VOL_AVG_10': [63758, 61525, 65968]},
 {'PRC_MEDIAN': [10675, 10600, 10475]},
 {'VOL_MEDIAN': [60328, 56974, 56974]},
 {'REMAIN_DEAL': [-29.0, nan, nan]}]

In [25]:
for idx, val in dict_col_index.items():
    print(idx, val)

FRGN_RT 0
END_PRC 1
VOL 2
REMAIN_BUY 3
REMAIN_SELL 4
REMAIN_BUY_AMOUNT 5
PRC_AVG_5 6
VOL_AVG_5 7
PRC_AVG_10 8
VOL_AVG_10 9
PRC_MEDIAN 10
VOL_MEDIAN 11
REMAIN_DEAL 12


In [14]:
# 재무제표 선정 종목
df_balance = pd.read_csv(f"{root_dir}balance_sheet.csv", encoding="utf-8-sig")

In [16]:
# 대상 종목의 외국인 추세 추가
df_balance["FRGN_RT"] = df_balance["NM"].apply(lambda X: dict_rising[X][0]['END_PRC'])

df_balance

KeyError: 'END_PRC'

In [14]:
import numpy as np
# 등락 비율
list_down_rt = []
list_up_rt = []
list_val = [58500, 51000, 59500, 61300, 61500]

for idx in range(len(list_val)-1):
    pre_prc = list_val[idx]
    next_prc = list_val[idx+1]
    if next_prc > pre_prc:
        list_up_rt.append((next_prc-pre_prc) / pre_prc)
    else:
        list_down_rt.append((pre_prc-next_prc) / next_prc)
        
print(round(np.average(list_up_rt) * 100, 2), round(np.average(list_down_rt) * 100, 2))

6.67 14.71


In [25]:
import pandas as pd

df_balance_sheet = pd.read_csv("./bs_analysis/balance_sheet.csv", encoding="utf-8-sig")

df_balance_sheet.loc[df_balance_sheet.CD == "A026890", 'NM'] = "스틱인베스트먼트"

df_balance_sheet[df_balance_sheet["CD"] == "A026890"]

,CD,NM,ROE,PER,PBR,PROFIT,INCOME
80,A026890,스틱인베스트먼트,"[7.12, 11.76, 28.35]","[20.2, 18.97, 10.57]","[1.25, 1.88, 2.41]","[169, 307, 763]","[111, 200, 581]"


In [26]:
df_balance_sheet.to_csv("./bs_analysis/balance_sheet.csv", encoding="utf-8-sig", index=False)